In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import eli5
import pickle

sns.set()
warnings.filterwarnings('ignore')

from sklearn.feature_selection import mutual_info_regression, SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier, RandomForestRegressor, RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, scale, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_absolute_percentage_error, mean_squared_error, roc_auc_score, log_loss, precision_recall_fscore_support, mean_absolute_error, plot_roc_curve
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from imblearn.over_sampling import RandomOverSampler, SMOTE
from eli5.sklearn import PermutationImportance
from pprint import pprint

from xgboost import XGBRegressor, XGBClassifier

In [2]:
data=pd.read_csv('Bondora_raw.csv')

In [3]:
data.head().T

,0,1,2,3,4
ReportAsOfEOD,2020-01-27,2020-01-27,2020-01-27,2020-01-27,2020-01-27
LoanId,F0660C80-83F3-4A97-8DA0-9C250112D6EC,978BB85B-1C69-4D51-8447-9C240104A3A2,EA44027E-7FA7-4BB2-846D-9C1F013C8A22,CE67AD25-2951-4BEE-96BD-9C2700C61EF4,9408BF8C-B159-4D6A-9D61-9C2400A986E3
LoanNumber,659,654,641,668,652
ListedOnUTC,2009-06-11 16:40:39,2009-06-10 15:48:57,2009-06-05 19:12:29,2009-06-13 12:01:20,2009-06-10 10:17:13
BiddingStartedOn,2009-06-11 16:40:39,2009-06-10 15:48:57,2009-06-05 19:12:29,2009-06-13 12:01:20,2009-06-10 10:17:13
...,...,...,...,...,...
NrOfScheduledPayments,NaN,NaN,NaN,NaN,NaN
ReScheduledOn,NaN,NaN,NaN,NaN,NaN
PrincipalDebtServicingCost,0.0,0.0,0.0,0.0,0.0
InterestAndPenaltyDebtServicingCost,0.0,0.0,0.0,0.0,0.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134529 entries, 0 to 134528
Columns: 112 entries, ReportAsOfEOD to ActiveLateLastPaymentCategory
dtypes: bool(3), float64(56), int64(14), object(39)
memory usage: 112.3+ MB


In [5]:
drop_cols=['ReportAsOfEOD',
 'LoanId',
 'LoanNumber',
 'ListedOnUTC','DateOfBirth',
 'BiddingStartedOn','UserName', 'LanguageCode','LoanApplicationStartedDate','FirstPaymentDate','IncomeFromPrincipalEmployer', 'IncomeFromPension',
       'IncomeFromFamilyAllowance', 'IncomeFromSocialWelfare',
       'IncomeFromLeavePay', 'IncomeFromChildSupport', 'IncomeOther','LoanApplicationStartedDate','ApplicationSignedHour',
       'ApplicationSignedWeekday','ActiveScheduleFirstPaymentReached','ModelVersion','WorseLateCategory','PlannedPrincipalTillDate',"NextPaymentNr",'ProbabilityOfDefault',
          'ExpectedLoss', 'LossGivenDefault','ExpectedReturn'] 

print(len(drop_cols))
data.drop(drop_cols,axis=1,inplace=True)

29


In [6]:
data.head()

,BidsPortfolioManager,BidsApi,BidsManual,NewCreditCustomer,LoanDate,ContractEndDate,MaturityDate_Original,MaturityDate_Last,VerificationType,Age,...,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,GracePeriodStart,GracePeriodEnd,NextPaymentDate,NrOfScheduledPayments,ReScheduledOn,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,ActiveLateLastPaymentCategory
0,0,0,115.0410,True,2009-06-16,2010-07-06,2010-06-25,2010-06-25,2.0,61,...,0.0,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1,0,0,140.6057,False,2009-06-15,2009-07-07,2009-07-15,2009-07-15,2.0,48,...,0.0,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
2,0,0,319.5580,True,2009-06-15,NaN,2011-02-25,2014-05-13,2.0,58,...,0.0,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,180+
3,0,0,57.5205,True,2009-06-15,2010-09-15,2010-09-15,2010-09-15,2.0,23,...,0.0,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
4,0,0,319.5582,True,2009-06-14,2016-07-28,2010-06-25,2010-06-25,2.0,25,...,0.0,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,180+


In [7]:
data.isnull().sum()

BidsPortfolioManager                       0
BidsApi                                    0
BidsManual                                 0
NewCreditCustomer                          0
LoanDate                                   0
                                       ...  
NrOfScheduledPayments                  36741
ReScheduledOn                          94207
PrincipalDebtServicingCost             75400
InterestAndPenaltyDebtServicingCost    75400
ActiveLateLastPaymentCategory          82279
Length: 84, dtype: int64

In [8]:
nan=data.isnull().sum()[data.isnull().sum()>0]
print(nan/len(data))
nan=nan/len(data)
drop_cols=list(nan[nan>0.5].index)
print(len(drop_cols))

ContractEndDate                        0.561559
VerificationType                       0.000335
Gender                                 0.000335
MonthlyPayment                         0.049692
County                                 0.273844
City                                   0.072802
Education                              0.000335
MaritalStatus                          0.000335
NrOfDependants                         0.735373
EmploymentStatus                       0.001464
EmploymentDurationCurrentEmployer      0.006512
EmploymentPosition                     0.731731
WorkExperience                         0.728520
OccupationArea                         0.000639
HomeOwnershipType                      0.012280
DebtToIncome                           0.000335
FreeCash                               0.000335
PlannedInterestTillDate                0.029302
LastPaymentOn                          0.070847
CurrentDebtDaysPrimary                 0.639349
DebtOccuredOn                          0

In [9]:
data.columns 

Index(['BidsPortfolioManager', 'BidsApi', 'BidsManual', 'NewCreditCustomer',
       'LoanDate', 'ContractEndDate', 'MaturityDate_Original',
       'MaturityDate_Last', 'VerificationType', 'Age', 'Gender', 'Country',
       'AppliedAmount', 'Amount', 'Interest', 'LoanDuration', 'MonthlyPayment',
       'County', 'City', 'UseOfLoan', 'Education', 'MaritalStatus',
       'NrOfDependants', 'EmploymentStatus',
       'EmploymentDurationCurrentEmployer', 'EmploymentPosition',
       'WorkExperience', 'OccupationArea', 'HomeOwnershipType', 'IncomeTotal',
       'ExistingLiabilities', 'LiabilitiesTotal', 'RefinanceLiabilities',
       'DebtToIncome', 'FreeCash', 'MonthlyPaymentDay',
       'PlannedInterestTillDate', 'LastPaymentOn', 'CurrentDebtDaysPrimary',
       'DebtOccuredOn', 'CurrentDebtDaysSecondary',
       'DebtOccuredOnForSecondary', 'DefaultDate',
       'PrincipalOverdueBySchedule', 'PlannedPrincipalPostDefault',
       'PlannedInterestPostDefault', 'EAD1', 'EAD2', 'PrincipalRecov

In [10]:
date_cols=["LoanDate","MaturityDate_Original","MaturityDate_Last",'LastPaymentOn']
data.drop(date_cols,axis=1,inplace=True)

In [11]:
status_current = data.loc[data["Status"]=="Current"]
status_without_current = data.drop(data.index[data['Status']=='Current'],inplace=True)

In [12]:
data["DefaultLoan"]=np.nan
defaultNAN=data.loc[data["DefaultDate"].isnull()]
defaultPRESENT=data.DefaultDate.drop(defaultNAN.index)
data["DefaultLoan"][defaultNAN.index]=0
data["DefaultLoan"][defaultPRESENT.index]=1

In [13]:
data['DefaultLoan'].value_counts()

1.0    42794
0.0    34600
Name: DefaultLoan, dtype: int64

In [14]:
data

,BidsPortfolioManager,BidsApi,BidsManual,NewCreditCustomer,ContractEndDate,VerificationType,Age,Gender,Country,AppliedAmount,...,PreviousEarlyRepaymentsCountBeforeLoan,GracePeriodStart,GracePeriodEnd,NextPaymentDate,NrOfScheduledPayments,ReScheduledOn,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,ActiveLateLastPaymentCategory,DefaultLoan
0,0,0,115.0410,True,2010-07-06,2.0,61,1.0,EE,319.5582,...,0,NaN,NaN,NaN,NaN,NaN,0.0,0.00,NaN,0.0
1,0,0,140.6057,False,2009-07-07,2.0,48,1.0,EE,191.7349,...,0,NaN,NaN,NaN,NaN,NaN,0.0,0.00,NaN,0.0
2,0,0,319.5580,True,NaN,2.0,58,1.0,EE,319.5582,...,0,NaN,NaN,NaN,NaN,NaN,0.0,0.00,180+,1.0
3,0,0,57.5205,True,2010-09-15,2.0,23,1.0,EE,127.8233,...,0,NaN,NaN,NaN,NaN,NaN,0.0,0.00,NaN,0.0
4,0,0,319.5582,True,2016-07-28,2.0,25,1.0,EE,319.5582,...,0,NaN,NaN,NaN,NaN,NaN,0.0,0.00,180+,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134524,2515,0,485.0000,True,2019-02-13,3.0,37,0.0,FI,3000.0000,...,0,NaN,NaN,NaN,NaN,NaN,0.0,1040.06,180+,1.0
134525,1880,0,1120.0000,False,2019-02-07,3.0,35,0.0,FI,3000.0000,...,0,2017-06-27,2018-06-20,NaN,NaN,NaN,0.0,0.00,NaN,0.0
134526,1975,0,525.0000,True,NaN,1.0,40,0.0,FI,2500.0000,...,0,NaN,NaN,NaN,60.0,NaN,0.0,1051.85,8-15,1.0
134527,1840,0,1160.0000,False,2019-12-23,4.0,47,1.0,EE,3000.0000,...,0,2016-04-01,2016-10-03,NaN,NaN,NaN,0.0,889.50,31-60,1.0


In [15]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [16]:
data['Education']= le.fit_transform(data['Education'])
data['EmploymentStatus']= le.fit_transform(data['EmploymentStatus'])
data['Gender']= le.fit_transform(data['Gender'])
data['HomeOwnershipType']= le.fit_transform(data['HomeOwnershipType'])
data['MaritalStatus']= le.fit_transform(data['MaritalStatus'])
data['VerificationType']= le.fit_transform(data['VerificationType'])
data['UseOfLoan']= le.fit_transform(data['UseOfLoan'])
data['OccupationArea']= le.fit_transform(data['OccupationArea'])
data['Country']= le.fit_transform(data['Country'])
data['NewCreditCustomer']= le.fit_transform(data['NewCreditCustomer'])
data['County']= le.fit_transform(data['County'])
data['City']= le.fit_transform(data['City'])
data['EmploymentDurationCurrentEmployer']= le.fit_transform(data['EmploymentDurationCurrentEmployer'])
data['RecoveryStage']= le.fit_transform(data['RecoveryStage'])
data['Rating']= le.fit_transform(data['Rating'])
data['DefaultLoan']= le.fit_transform(data['DefaultLoan'])
data['Restructured']= le.fit_transform(data['Restructured'])
data['CreditScoreEsMicroL']= le.fit_transform(data['CreditScoreEsMicroL'])

In [17]:
# data.loc[data['NewCreditCustomer'] == False,'NewCreditCustomer'] = 'Existing_credit_customer'
# data.loc[data['NewCreditCustomer'] == True,'NewCreditCustomer'] = 'New_credit_Customer'

# data.loc[data['Restructured'] == False,'Restructured']='No'
# data.loc[data['Restructured'] == True,'Restructured']='Yes'

# data.loc[data['RecoveryStage']==1,'RecoveryStage']='Collection'
# data.loc[data['RecoveryStage']==0,'RecoveryStage']='Recovery'

In [17]:
numerical_cols=[col for col in data.columns if data[col].dtype!=object]
categorical_cols=[col for col in data.columns if data[col].dtype==object]
print("No of Numerical features =",len(numerical_cols))
print("No of Categorical features =",len(categorical_cols))

No of Numerical features = 61
No of Categorical features = 20


In [18]:
for i in categorical_cols:
    data[i] = data[i].fillna(data[i].mode()[0])

In [19]:
for i in numerical_cols:
    data[i] = data[i].fillna(data[i].median())

In [20]:
data.isnull().sum()

BidsPortfolioManager                   0
BidsApi                                0
BidsManual                             0
NewCreditCustomer                      0
ContractEndDate                        0
                                      ..
ReScheduledOn                          0
PrincipalDebtServicingCost             0
InterestAndPenaltyDebtServicingCost    0
ActiveLateLastPaymentCategory          0
DefaultLoan                            0
Length: 81, dtype: int64

In [21]:

def drop_outliers_zscore(dfcopy:pd.DataFrame, cols, threshold:int=3, inplace:bool=False):
    
    if inplace:
        global data
    else:
        data = dfcopy.copy()

    def drop_col(df_, col):
        
        mean, std = np.mean(df_[col]), np.std(df_[col])
        if std==0:
            std = 0.1
        df_['is_outlier'] = df_[col].apply(lambda x : np.abs((x - mean) / std) > threshold)
        outliers_idx = df_.loc[df_['is_outlier']].index
        df_ = df_.drop(outliers_idx, axis=0)
        
        data = df_.drop('is_outlier', axis=1)
        return data

    
    if type(cols) == str:
        data = drop_col(data, cols)
    elif type(cols) == list:
        for col in cols:
            data = drop_col(data, col)
    else :
        raise ValueError('Pass neither list nor string in {Cols}')
    
    if inplace:
        
        dfcopy = data
    else:
        return data
    
THRESHOLD = 4
num_outlier_records =  data.shape[0] - drop_outliers_zscore(data,numerical_cols , threshold=THRESHOLD).shape[0]
print(f'Number of Outliers {num_outlier_records}, with threshold, with threshold {THRESHOLD}')

drop_outliers_zscore(data, numerical_cols, threshold=THRESHOLD, inplace=True)

Number of Outliers 23121, with threshold, with threshold 4


In [22]:
%%time
start='\033[1m'
end= '\033[0;0m'

CPU times: total: 0 ns
Wall time: 0 ns


In [23]:
print(start+"Shape of the Dataset:"+end,data.shape,"\n")

Shape of the Dataset: (38820, 81) 



In [24]:
data=data[data['Age']>10]
data=data[data['AppliedAmount']>0]
data=data[data['DebtToIncome']<100]
data=data[data['CreditScoreEeMini']!=0]
data=data[data['PrincipalBalance']>=0]

In [25]:
data.Status.unique()

array(['Repaid', 'Late'], dtype=object)

In [26]:
data_drop=['ContractEndDate','County','City','NrOfDependants', 'EmploymentPosition','WorkExperience','MonthlyPaymentDay','PlannedInterestTillDate','CurrentDebtDaysPrimary', 'DebtOccuredOn',
       'CurrentDebtDaysSecondary', 'DebtOccuredOnForSecondary','PrincipalOverdueBySchedule', 'PlannedPrincipalPostDefault',
       'PlannedInterestPostDefault', 'EAD1', 'EAD2', 'PrincipalRecovery',
       'InterestRecovery', 'RecoveryStage', 'StageActiveSince','EL_V0', 'Rating_V0', 'EL_V1', 'Rating_V1', 'Rating_V2', 'Status','Restructured', 'ActiveLateCategory','CreditScoreEsEquifaxRisk', 'CreditScoreFiAsiakasTietoRiskGrade',
       'CreditScoreEeMini','PrincipalWriteOffs',
       'InterestAndPenaltyWriteOffs', 'PrincipalBalance',
       'InterestAndPenaltyBalance', 'NoOfPreviousLoansBeforeLoan',
       'AmountOfPreviousLoansBeforeLoan', 'PreviousRepaymentsBeforeLoan','GracePeriodStart',
       'GracePeriodEnd', 'NextPaymentDate', 'NrOfScheduledPayments',
       'ReScheduledOn', 'PrincipalDebtServicingCost',
       'InterestAndPenaltyDebtServicingCost', 'ActiveLateLastPaymentCategory','DefaultDate']
print(len(data_drop))
data.drop(data_drop,axis=1,inplace=True)

48


In [27]:
data.head(2)

,BidsPortfolioManager,BidsApi,BidsManual,NewCreditCustomer,VerificationType,Age,Gender,Country,AppliedAmount,Amount,...,RefinanceLiabilities,DebtToIncome,FreeCash,Rating,CreditScoreEsMicroL,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,DefaultLoan
5,0,0,300.3845,1,2,22,0,0,300.3847,300.4314,...,0,0.0,0.0,8,11,300.4314,124.2294,0.0,0,1
9,0,0,319.5581,1,2,38,1,0,319.5582,319.5583,...,0,0.0,0.0,8,11,319.5583,45.4153,0.0,0,0


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38817 entries, 5 to 134525
Data columns (total 33 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   BidsPortfolioManager                    38817 non-null  int64  
 1   BidsApi                                 38817 non-null  int64  
 2   BidsManual                              38817 non-null  float64
 3   NewCreditCustomer                       38817 non-null  int64  
 4   VerificationType                        38817 non-null  int64  
 5   Age                                     38817 non-null  int64  
 6   Gender                                  38817 non-null  int64  
 7   Country                                 38817 non-null  int32  
 8   AppliedAmount                           38817 non-null  float64
 9   Amount                                  38817 non-null  float64
 10  Interest                                38817 non-null  f

In [29]:
#Preferred loan amount(AppliedAmount,Amount,)
# Preferred emi(LoanDuration,MonthlyPayment)
# Preferred Roi(Interest)
#Preferred loan amount=data["Amount"]



In [30]:
data_temp =data[['LoanDuration', 'Interest', 'Amount']]
data_temp.head()

,LoanDuration,Interest,Amount
5,24,30.0,300.4314
9,12,25.0,319.5583
10,24,25.0,319.4988
26,3,20.0,319.5587
31,4,25.0,255.6467


In [31]:
def cal_EMI(P, r, n):
  P = P.values
  r = r.values
  n = n.values
  #print(P.shape[0])
  result_1 = np.empty(0)
  result_2 = np.empty(0)
  result = np.empty(0)
  for i in range(P.shape[0]):
    #print(P[i])
    #print(r[i])
    #print(n[i])
    # EMI = P × r × (1 + r) ^ n / ((1 + r) ^ n – 1)
    #print(P[i] * (1 + r[i]))
    result_1 = np.append(result_1, P[i] * r[i] * np.power((1 + r[i]),n[i]))
    result_2 = np.append(result_2, np.power((1 + r[i]),n[i]) - 1)
    result = np.append(result, (result_1[i] / result_2[i]))

  return result

In [32]:
data_temp['EMI'] = cal_EMI(data_temp['Amount'],data_temp['Interest'],data_temp['LoanDuration'])

In [33]:
data['EMI'] = data_temp['EMI']

In [34]:
data['EMI'].head()

5     9012.942000
9     7988.957500
10    7987.470000
26    6391.864192
31    6391.181486
Name: EMI, dtype: float64

In [35]:
data_temp = data[['AppliedAmount', 'Interest', 'IncomeTotal', 'LiabilitiesTotal', 'LoanDuration']]
data_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38817 entries, 5 to 134525
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AppliedAmount     38817 non-null  float64
 1   Interest          38817 non-null  float64
 2   IncomeTotal       38817 non-null  float64
 3   LiabilitiesTotal  38817 non-null  float64
 4   LoanDuration      38817 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 1.8 MB


In [36]:
data_temp[data_temp['IncomeTotal']==3665].shape

(0, 5)

In [37]:
# Step 1
data_temp['Ava_Inc'] = ((data_temp['IncomeTotal']-data_temp['LiabilitiesTotal'])*0.3)
data_temp['Total_Loan_Amnt'] = np.round((data['AppliedAmount'] + (data['AppliedAmount'] * data['Interest']) /100)*data['LoanDuration'])
data_temp.head()

,AppliedAmount,Interest,IncomeTotal,LiabilitiesTotal,LoanDuration,Ava_Inc,Total_Loan_Amnt
5,300.3847,30.0,9500.0,0.0,24,2850.0,9372.0
9,319.5582,25.0,7700.0,0.0,12,2310.0,4793.0
10,319.5582,25.0,4080.0,0.0,24,1224.0,9587.0
26,319.5582,20.0,9887.0,0.0,3,2966.1,1150.0
31,255.6466,25.0,9000.0,0.0,4,2700.0,1278.0


In [38]:
# Step 2
def eligible_loan_amnt(df):
  Ava_Inc = df['Ava_Inc'].values
  Total_Loan_Amnt = df['Total_Loan_Amnt'].values
  ELA = np.empty(0)
  for i in range(len(Ava_Inc)):
    if Total_Loan_Amnt[i] <= Ava_Inc[i]:
      ELA = np.append(ELA, Total_Loan_Amnt[i])
    else:
      ELA = np.append(ELA, Ava_Inc[i])
  return ELA

In [39]:
data_temp['ELA'] = eligible_loan_amnt(data_temp)

In [40]:
data_temp.head()

,AppliedAmount,Interest,IncomeTotal,LiabilitiesTotal,LoanDuration,Ava_Inc,Total_Loan_Amnt,ELA
5,300.3847,30.0,9500.0,0.0,24,2850.0,9372.0,2850.0
9,319.5582,25.0,7700.0,0.0,12,2310.0,4793.0,2310.0
10,319.5582,25.0,4080.0,0.0,24,1224.0,9587.0,1224.0
26,319.5582,20.0,9887.0,0.0,3,2966.1,1150.0,1150.0
31,255.6466,25.0,9000.0,0.0,4,2700.0,1278.0,1278.0


In [41]:
data['ELA'] = data_temp['ELA']
data.columns

Index(['BidsPortfolioManager', 'BidsApi', 'BidsManual', 'NewCreditCustomer',
       'VerificationType', 'Age', 'Gender', 'Country', 'AppliedAmount',
       'Amount', 'Interest', 'LoanDuration', 'MonthlyPayment', 'UseOfLoan',
       'Education', 'MaritalStatus', 'EmploymentStatus',
       'EmploymentDurationCurrentEmployer', 'OccupationArea',
       'HomeOwnershipType', 'IncomeTotal', 'ExistingLiabilities',
       'LiabilitiesTotal', 'RefinanceLiabilities', 'DebtToIncome', 'FreeCash',
       'Rating', 'CreditScoreEsMicroL', 'PrincipalPaymentsMade',
       'InterestAndPenaltyPaymentsMade', 'PreviousEarlyRepaymentsBefoleLoan',
       'PreviousEarlyRepaymentsCountBeforeLoan', 'DefaultLoan', 'EMI', 'ELA'],
      dtype='object')

In [42]:
data_temp = data[['Amount', 'Interest']]
data_temp.head()

,Amount,Interest
5,300.4314,30.0
9,319.5583,25.0
10,319.4988,25.0
26,319.5587,20.0
31,255.6467,25.0


In [43]:
data_temp['InterestAmount'] = (data_temp['Amount']*(data_temp['Interest']/100))
data_temp['TotalAmount'] = (data_temp['InterestAmount'] + data_temp['Amount'])
data_temp['ROI'] = (data_temp['InterestAmount'] / data_temp['TotalAmount'])*100
data['ROI'] = data_temp['ROI']

In [44]:
data_temp.head()

,Amount,Interest,InterestAmount,TotalAmount,ROI
5,300.4314,30.0,90.129420,390.560820,23.076923
9,319.5583,25.0,79.889575,399.447875,20.000000
10,319.4988,25.0,79.874700,399.373500,20.000000
26,319.5587,20.0,63.911740,383.470440,16.666667
31,255.6467,25.0,63.911675,319.558375,20.000000


In [45]:
data

,BidsPortfolioManager,BidsApi,BidsManual,NewCreditCustomer,VerificationType,Age,Gender,Country,AppliedAmount,Amount,...,Rating,CreditScoreEsMicroL,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,DefaultLoan,EMI,ELA,ROI
5,0,0,300.3845,1,2,22,0,0,300.3847,300.4314,...,8,11,300.4314,124.2294,0.0,0,1,9012.942000,2850.000,23.076923
9,0,0,319.5581,1,2,38,1,0,319.5582,319.5583,...,8,11,319.5583,45.4153,0.0,0,0,7988.957500,2310.000,20.000000
10,0,0,319.5581,1,2,54,1,0,319.5582,319.4988,...,8,11,319.4988,89.8738,0.0,0,1,7987.470000,1224.000,20.000000
26,0,0,319.5581,1,2,51,1,0,319.5582,319.5587,...,8,11,319.5587,14.2036,0.0,0,0,6391.864192,1150.000,16.666667
31,0,0,255.6464,1,2,24,0,0,255.6466,255.6467,...,8,11,255.6467,11.4975,0.0,0,0,6391.181486,1278.000,20.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134513,1500,0,0.0000,0,4,68,1,0,1500.0000,1500.0000,...,1,11,1500.0000,371.4200,0.0,0,0,22410.000000,109.983,12.998086
134516,1000,0,0.0000,1,4,27,1,0,1000.0000,1000.0000,...,3,11,1000.0000,359.1700,0.0,0,0,20800.000000,-27.900,17.218543
134517,1400,0,0.0000,0,4,25,0,0,1400.0000,1400.0000,...,2,11,1400.0000,65.5600,0.0,0,0,24962.000000,246.657,15.131970
134519,2000,0,0.0000,1,1,38,0,2,2000.0000,2000.0000,...,4,11,906.6500,1130.6900,0.0,0,1,47480.000000,10.500,19.185389


In [46]:
# Let's compute IQR for each numerical feature
df_IQR = data[data.select_dtypes([float, int]).columns].quantile(.75) - data[data.select_dtypes([float, int]).columns].quantile(.25)

In [47]:
# Let's compute maximum and minimum limits
df_Max =  data[data.select_dtypes([float, int]).columns].quantile(.75) + (1.5*df_IQR)
df_Min =  data[data.select_dtypes([float, int]).columns].quantile(.25) - (1.5*df_IQR)

In [48]:
data.select_dtypes([float, int]).columns

Index(['BidsPortfolioManager', 'BidsApi', 'BidsManual', 'NewCreditCustomer',
       'VerificationType', 'Age', 'Gender', 'Country', 'AppliedAmount',
       'Amount', 'Interest', 'LoanDuration', 'MonthlyPayment', 'UseOfLoan',
       'Education', 'MaritalStatus', 'EmploymentStatus',
       'EmploymentDurationCurrentEmployer', 'OccupationArea',
       'HomeOwnershipType', 'IncomeTotal', 'ExistingLiabilities',
       'LiabilitiesTotal', 'RefinanceLiabilities', 'DebtToIncome', 'FreeCash',
       'Rating', 'CreditScoreEsMicroL', 'PrincipalPaymentsMade',
       'InterestAndPenaltyPaymentsMade', 'PreviousEarlyRepaymentsBefoleLoan',
       'PreviousEarlyRepaymentsCountBeforeLoan', 'DefaultLoan', 'EMI', 'ELA',
       'ROI'],
      dtype='object')

In [49]:
col_IQR = data['Age'].quantile(.75) - data['Age'].quantile(.25)
col_Max = data['Age'].quantile(.75) + (1.5*col_IQR)

In [50]:
# Loop for replacing outliers above upper bound with the upper bound value:
for column in data.select_dtypes([float, int]).columns :
   
    col_IQR = data[column].quantile(.75) - data[column].quantile(.25)
    col_Max = data[column].quantile(.75) + (1.5*col_IQR)
    data[column][data[column] > col_Max] =  col_Max

In [51]:
# Loop for replacing outliers under lower bound with the lower bound value:
for column in data.select_dtypes([float, int]).columns :
    col_IQR = data[column].quantile(.75) - data[column].quantile(.25)
    col_Min = data[column].quantile(.25) - (1.5*col_IQR)
    data[column][data[column] < col_Min] =  col_Min

## Pipeline 

In [52]:
X=data.iloc[0:,:-3]

In [53]:
X.head(2)


,BidsPortfolioManager,BidsApi,BidsManual,NewCreditCustomer,VerificationType,Age,Gender,Country,AppliedAmount,Amount,...,RefinanceLiabilities,DebtToIncome,FreeCash,Rating,CreditScoreEsMicroL,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,DefaultLoan
5,0.0,0,300.3845,1,2,22,0,0,300.3847,300.4314,...,0,0.0,0.0,8,2.5,300.4314,124.2294,0.0,0,1
9,0.0,0,319.5581,1,2,38,1,0,319.5582,319.5583,...,0,0.0,0.0,8,2.5,319.5583,45.4153,0.0,0,0


In [54]:
y=data.iloc[:,-3:]
y

,EMI,ELA,ROI
5,9012.942000,878.8905,23.076923
9,7988.957500,878.8905,20.000000
10,7987.470000,878.8905,20.000000
26,6391.864192,878.8905,16.666667
31,6391.181486,878.8905,20.000000
...,...,...,...
134513,22410.000000,109.9830,12.998086
134516,20800.000000,-27.9000,17.218543
134517,24962.000000,246.6570,15.131970
134519,47480.000000,10.5000,19.185389


In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [56]:
over = RandomOverSampler(random_state=0)
smote = SMOTE()

stdscaler = StandardScaler()
scaler = MinMaxScaler()

pca = PCA(n_components=2)

In [ ]:
LinearDiscriminantAnalysis
QuadraticDiscriminantAnalysis

In [58]:
#LG,LOG-G,Rand_F,Ridge

In [59]:
#Logistic Regression and Random Forest Classifier Regression Models (1. linear Reggresion 2. Ridge Regression L2 Norm).

In [57]:
Lin_G=LinearRegression()
ridge= Ridge(alpha=0.1)
Rand_F = RandomForestRegressor(random_state=0)

stdscaler= StandardScaler

In [67]:
steps = [('stdscaler', StandardScaler()),('pca', PCA()),('Lin_G', LinearRegression()), ('ridge', Ridge()),("Rand_F",RandomForestRegressor())]
#clf = Pipeline(["S"])

In [99]:
model_rand_F=[('stdscaler', StandardScaler()),('pca', PCA()),("Rand_F",RandomForestRegressor())]
pipe=Pipeline(st)
pipe

Pipeline(steps=[('stdscaler', StandardScaler()), ('pca', PCA()),
                ('Rand_F', RandomForestRegressor())])

In [100]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('stdscaler', StandardScaler()), ('pca', PCA()),
                ('Rand_F', RandomForestRegressor())])

In [92]:
pipe.predict(X_test)

array([[5.43898010e+04, 1.66516170e+02, 2.80954295e+01],
       [7.61273235e+04, 3.32544885e+02, 2.36206464e+01],
       [6.49310230e+04, 1.73281125e+02, 2.25084730e+01],
       ...,
       [9.08095808e+04, 4.95108360e+02, 2.14151064e+01],
       [1.99744467e+04, 2.41065720e+02, 1.76062519e+01],
       [6.98840871e+04, 3.84388110e+02, 2.46097455e+01]])

In [94]:
#pipe.score(X_train,y_train)
print('test accuracy = ', round(pipe.score(X_train,y_train)*100,2), "%")


test accuracy =  98.09 %


In [72]:
from sklearn import set_config
set_config(display="diagram")
steps

[('QDA', QuadraticDiscriminantAnalysis()),
 ('LDA', LinearDiscriminantAnalysis()),
 ('Lin_G', LinearRegression()),
 ('ridge', Ridge()),
 ('Rand_F', RandomForestRegressor())]

In [101]:
model_lin_R=[('stdscaler', StandardScaler()),('pca', PCA()),("LIN",LinearRegression())]
pip=Pipeline(sa)
pip

Pipeline(steps=[('stdscaler', StandardScaler()), ('pca', PCA()),
                ('LIN', LinearRegression())])

In [102]:
pip.fit(X_train,y_train)

Pipeline(steps=[('stdscaler', StandardScaler()), ('pca', PCA()),
                ('LIN', LinearRegression())])

In [103]:
pip.predict(X_test)

array([[6.68215994e+04, 1.12338486e+02, 3.43323371e+01],
       [9.06858736e+04, 4.29218188e+02, 2.91842431e+01],
       [5.64906845e+04, 1.16932997e+02, 1.98090197e+01],
       ...,
       [9.99821484e+04, 5.70627314e+02, 1.74107361e+01],
       [2.17710485e+04, 9.20837713e+01, 1.72508168e+01],
       [8.03023841e+04, 4.93325496e+02, 2.43117810e+01]])

In [104]:
#pip.score(X_train,y_train)
print('test accuracy = ', round(pip.score(X_train,y_train)*100,2), "%")


test accuracy =  95.93 %


In [105]:
model_ridge=[('stdscaler', StandardScaler()),('pca', PCA()),("ridge",Ridge())]
pipe1=Pipeline(sb)

pipe1

Pipeline(steps=[('stdscaler', StandardScaler()), ('pca', PCA()),
                ('ridge', Ridge())])

In [106]:
pipe1.fit(X_train,y_train)

Pipeline(steps=[('stdscaler', StandardScaler()), ('pca', PCA()),
                ('ridge', Ridge())])

In [108]:
pipe1.predict(X_test)


array([[6.68210350e+04, 1.12343973e+02, 3.43325909e+01],
       [9.06856320e+04, 4.29203322e+02, 2.91839519e+01],
       [5.64882456e+04, 1.16945542e+02, 1.98091915e+01],
       ...,
       [9.99858679e+04, 5.70618974e+02, 1.74112046e+01],
       [2.17695819e+04, 9.20868403e+01, 1.72510511e+01],
       [8.03065853e+04, 4.93320329e+02, 2.43124561e+01]])

In [109]:
print('test accuracy = ', round(pipe1.score(X_train,y_train)*100,2), "%")

test accuracy =  95.93 %


In [118]:
import joblib 
joblib.dump(model_rand_F,"Random_Regresser")
joblib.dump(model_ridge,"ridge_Regresser")
joblib.dump(model_lin_R,"Linear_Regresser")

['Linear_Regresser']